# VRBO Scrape

In [58]:
import requests
from bs4 import BeautifulSoup
import csv
from urllib.parse import urlparse, parse_qs
import os


# URL of the page to scrape
url = "https://www.vrbo.com/search?adults=2&children=&regionId=6233839&destination=Southwest%20Florida%2C%20Florida%2C%20United%20States%20of%20America&latLong=26.682838%2C-81.795225&chkin=2024-12-01&chkout=2025-01-31&d1=2024-12-01&d2=2025-01-31&startDate=2024-12-01&endDate=2025-01-31&privacyTrackingState=CAN_TRACK&searchId=bae94e74-c608-4667-ad76-eb74864c5446&searchIdTest=test&sort=RECOMMENDED&useRewards=SHOP_WITH_POINTS&theme=&pwaDialog=&semdtl=&userIntent="
#url = "https://www.vrbo.com/search?destination=43111%20Greenway%20Blvd%2C%20Punta%20Gorda%2C%20FL%2033982%2C%20USA&flexibility=0_DAY&d1=2024-11-01&startDate=2024-11-01&d2=2024-12-31&endDate=2024-12-31&adults=2&theme=&userIntent=&semdtl=&sort=RECOMMENDED"
#url = "https://www.vrbo.com/search?destination=Naples%2C%20Florida%2C%20United%20States%20of%20America&d1=2024-11-01&startDate=2025-01-31&d2=2024-12-31&endDate=2025-02-01&adults=2&theme=&userIntent=&semdtl=&latLong=26.141261%2C-81.794586&regionId=602724&children=&pwaDialog=&mapBounds=&allowPreAppliedFilters=false&amenities=&sort=RECOMMENDED"
#url = "https://www.vrbo.com/search?destination=Fort%20Myers%2C%20Florida%2C%20United%20States%20of%20America&d1=2024-11-01&startDate=2024-12-01&d2=2024-12-31&endDate=2025-01-31&adults=2&theme=&userIntent=&semdtl=&latLong=26.640629%2C-81.872307&regionId=602729&children=&pwaDialog=&mapBounds=&allowPreAppliedFilters=false&sort=RECOMMENDED&amenities="
#url = "https://www.vrbo.com/search?destination=Cape%20Coral%2C%20Florida%2C%20United%20States%20of%20America&d1=2024-11-01&startDate=2024-12-01&d2=2024-12-31&endDate=2025-01-31&adults=2&theme=&userIntent=&semdtl=&latLong=26.562853%2C-81.949532&regionId=6521&children=&pwaDialog=&mapBounds=&allowPreAppliedFilters=true&amenities=&sort=RECOMMENDED"    
3url = "https://www.vrbo.com/search?destination=Bonita%20Springs%2C%20Florida%2C%20United%20States%20of%20America&d1=2024-11-01&startDate=2024-11-01&d2=2024-12-31&endDate=2024-12-31&adults=2&theme=&userIntent=&semdtl=&latLong=26.339806%2C-81.778694&regionId=6403&children=&pwaDialog=&mapBounds=&allowPreAppliedFilters=false&sort=RECOMMENDED&amenities="
#url = "https://www.vrbo.com/3894161?chkin=2024-11-01&chkout=2024-12-31&d1=2024-11-01&d2=2024-12-31&startDate=2024-11-01&endDate=2024-12-31&x_pwa=1&rfrr=HSR&pwa_ts=1721373179972&referrerUrl=aHR0cHM6Ly93d3cudnJiby5jb20vSG90ZWwtU2VhcmNo&useRewards=true&adults=2&destination=43111+Greenway+Blvd%2C+Punta+Gorda%2C+FL+33982%2C+USA&destType=ADDRESS&latLong=26.7948913%2C-81.7353078&bedroom_count_gt=3&nearby_activities_group=golfing&pricing_group=nightly_price&privacyTrackingState=CAN_TRACK&property_type_group=apartment_or_condo&searchId=067bab4e-10fa-4324-826e-0da44123aaf3&us_bathroom_count_gt=2&sort=RECOMMENDED&top_dp=125&top_cur=USD&userIntent=&selectedRoomType=103028702&selectedRatePlan=0004ebad9607617a43148548e503d1e4d3ea&expediaPropertyId=103028702&propertyName=Luxe+3-Bed+Condo+in+Babcock+National+Golf+%26+Country+Club+with+Golf+Membership"

# Send a GET request to the URL
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

# Find the property listings
properties = soup.find_all('a', {'data-stid': 'open-hotel-information'}, limit=50)

print(f"Found {len(properties)} properties")

# Check if the file already exists
file_exists = os.path.isfile('sw_florida_vrbo_properties.csv')

# Open a CSV file to write the data
#with open('sw_florida_vrbo_properties.csv', mode='w', newline='') as file:
with open('sw_florida_vrbo_properties.csv', mode='a', newline='') as file:

    writer = csv.writer(file)
    
        # Write the header only if the file does not exist
    if not file_exists:
        writer.writerow(['Property ID', 'Title', 'URL', 'Price', 'TotalPrice', 'Latitude', 'Longitude', 'LatLong', 'Rating Badge', 'Rating Description', 'Number of Reviews', 'Check-in Date', 'Check-out Date', 'Region ID', 'Destination', 'Price (top_dp)', 'Currency (top_cur)', 'Room Type', 'Rate Plan', 'Neighborhood ID', 'Privacy Tracking State', 'Search ID', 'User Intent', 'Referrer URL', 'PWA Timestamp', 'Sleeps', 'Bedrooms', 'Bathrooms'])

    # Loop through the properties and extract the required information
    for property in properties:
        title = property.find('span', class_='is-visually-hidden').text.strip() if property.find('span', class_='is-visually-hidden') else 'N/A'
        title = title.replace("More information about ", "").replace(", opens in a new tab", "")  # Remove the unwanted phrases
        property_url = property['href'] if 'href' in property.attrs else 'N/A'
        
        # Parse the URL to extract query parameters
        parsed_url = urlparse(property_url)
        query_params = parse_qs(parsed_url.query)
        
        property_id = parsed_url.path.split('/')[-1] if parsed_url.path else 'N/A'
        checkin_date = query_params.get('chkin', ['N/A'])[0]
        checkout_date = query_params.get('chkout', ['N/A'])[0]
        region_id = query_params.get('regionId', ['N/A'])[0]
        destination = query_params.get('destination', ['N/A'])[0]
        price_top_dp = query_params.get('top_dp', ['N/A'])[0]
        currency_top_cur = query_params.get('top_cur', ['N/A'])[0]
        room_type = query_params.get('selectedRoomType', ['N/A'])[0]
        rate_plan = query_params.get('selectedRatePlan', ['N/A'])[0]
        neighborhood_id = query_params.get('neighborhoodId', ['N/A'])[0]
        privacy_tracking_state = query_params.get('privacyTrackingState', ['N/A'])[0]
        search_id = query_params.get('searchId', ['N/A'])[0]
        user_intent = query_params.get('userIntent', ['N/A'])[0]
        referrer_url = query_params.get('referrerUrl', ['N/A'])[0]
        pwa_timestamp = query_params.get('pwa_ts', ['N/A'])[0]
        
        # Extract latitude and longitude from the URL
        lat_long = query_params.get('latLong', ['N/A'])[0].replace('%2C', ',')
        latitude, longitude = lat_long.split(',') if lat_long != 'N/A' else ('N/A', 'N/A')
        
        # Find the price using the specified path
        price_div = property.find_parent('div').find("div", {"data-test-id": "price-summary"}).find("span").find("div")
        price = price_div.text.strip() if price_div else 'N/A'
        
        # Find the total price
        total_price_div = property.find_parent('div').find("div", class_="uitk-layout-flex uitk-layout-flex-flex-direction-row-reverse").find("div", class_="uitk-text uitk-type-200 uitk-type-regular uitk-text-default-theme")
        total_price = total_price_div.text.strip() if total_price_div else 'N/A'
        total_price = total_price.replace(' total', '').strip()  # Remove ' total'
        
        
        # Find the section where rating information is located and print it
        parent_div = property.find_parent('div')
        #print("Parent div containing rating information:")
        #print(parent_div.prettify())
        
        # Find the rating badge, description, and number of reviews using provided CSS selectors
        rating_badge_tag = parent_div.find('span', class_='uitk-badge-base-text')
        rating_badge = rating_badge_tag.text.strip() if rating_badge_tag else 'N/A'

        rating_description_tag = parent_div.find('span', class_='uitk-text uitk-type-300 uitk-type-medium uitk-text-emphasis-theme')
        rating_description = rating_description_tag.text.strip() if rating_description_tag else 'N/A'

        number_of_reviews_tag = parent_div.find('span', class_='uitk-text uitk-type-200 uitk-type-regular uitk-text-default-theme')
        number_of_reviews = number_of_reviews_tag.text.strip().split()[0] if number_of_reviews_tag else 'N/A'
        
        # Extract Sleeps, Bedrooms, Bathrooms information
        details_div = parent_div.find("div", class_="uitk-text uitk-text-spacing-half truncate-lines-2 uitk-type-300 uitk-text-default-theme")
        if details_div:
            details_text = details_div.text
            sleeps = details_text.split("·")[0].strip().replace("Sleeps ", "") if "Sleeps" in details_text else 'N/A'
            bedrooms = details_text.split("·")[1].strip().replace(" bedrooms", "") if "bedrooms" in details_text else 'N/A'
            bathrooms = details_text.split("·")[2].strip().replace(" bathrooms", "") if "bathrooms" in details_text else 'N/A'
        else:
            sleeps, bedrooms, bathrooms = 'N/A', 'N/A', 'N/A'
        
        # Print and write the detailed information
        print(f"Property ID: {property_id}, Title: {title}, URL: {property_url}, Price: {price}, TotalPrice: {total_price}, Latitude: {latitude}, Longitude: {longitude}, LatLong: {lat_long}, Rating Badge: {rating_badge}, Rating Description: {rating_description}, Number of Reviews: {number_of_reviews}, Check-in Date: {checkin_date}, Check-out Date: {checkout_date}, Region ID: {region_id}, Destination: {destination}, Price (top_dp): {price_top_dp}, Currency (top_cur): {currency_top_cur}, Room Type: {room_type}, Rate Plan: {rate_plan}, Neighborhood ID: {neighborhood_id}, Privacy Tracking State: {privacy_tracking_state}, Search ID: {search_id}, User Intent: {user_intent}, Referrer URL: {referrer_url}, PWA Timestamp: {pwa_timestamp}, Sleeps: {sleeps}, Bedrooms: {bedrooms}, Bathrooms: {bathrooms}")
        writer.writerow([property_id, title, property_url, price, total_price, latitude, longitude, lat_long, rating_badge, rating_description, number_of_reviews, checkin_date, checkout_date, region_id, destination, price_top_dp, currency_top_cur, room_type, rate_plan, neighborhood_id, privacy_tracking_state, search_id, user_intent, referrer_url, pwa_timestamp, sleeps, bedrooms, bathrooms])
        
print("Data has been written/appended to sw_florida_vrbo_properties.csv")




Found 0 properties
Data has been written/appended to sw_florida_vrbo_properties.csv
